In [1]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import math
from tqdm import tqdm
import random
import itertools
import pickle as cPickle

### 마크다운 문서
pl player : Human player
p2 player : Policy player
possible actions =  [0, 1, 2, 3, 4, 5, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 3
+----+----+----+
|    |    |    |
+----+----+----+
|  O |    |    |
+----+----+----+
|    |    |    |
+----+----+----+
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |    |    |
+----+----+----+
|    |    |    |
+----+----+----+
possible actions =  [1, 2, 4, 5, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 5
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |    |  O |
+----+----+----+
|    |    |    |
+----+----+----+
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|    |    |    |
+----+----+----+
possible actions =  [1, 2, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 8
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|    |    |  O |
+----+----+----+
+----+----+----+
|  X |    |  X |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|    |    |  O |
+----+----+----+
possible actions =  [1, 6, 7]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 6
+----+----+----+
|  X |    |  X |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|  O |    |  O |
+----+----+----+
+----+----+----+
|  X |  X |  X |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|  O |    |  O |
+----+----+----+
winner is p2(Policy player)
final result
+----+----+----+
|  X |  X |  X |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|  O |    |  O |
+----+----+----+
More Game? (y/n)n
p1(Human player) = 0 p2(Policy player) = 1 draw = 0


### Tic Tac Toe 환경 정의

In [2]:
class Environment():
    def __init__(self):
        self.state = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.go = 1

    def set_reward(self, action):
        self.state[action] = self.go
        self.go = self.go % 2 + 1
        done, winner = end_check(self.state)
        reward = 0
        if done and winner == 1: 
            reward = 1
        if done and winner == 2: 
            reward = -1
        if done and winner == 0: 
            reward = 0

        return self.state, done, reward, winner

    def setting(self):
        self.state = [0, 0, 0, 0, 0, 0, 0, 0, 0]
        self.go = 1

    def print_board(self):
        print("+----+----+----+")
        for i in range(3):
            for j in range(3):
                if self.state[3 * i + j] == 1:
                    print("|  O",end=" ")
                elif self.state[3 * i + j] == 2:
                    print("|  X",end=" ")
                else:
                    print("|   ",end=" ")
            print("|")
            print("+----+----+----+")
            

def end_check(state):
    for i in range(3):
        if state[3 * i] == state[3 * i + 1] and state[3 * i + 1] == state[3 * i + 2] \
                and state[3 * i] != 0:
            return True, state[3 * i]
        if state[i] == state[3 + i] and state[3 + i] == state[6 + i] and state[i] != 0:
            return True, state[i]
    if state[0] == state[4] and state[4] == state[8] and state[0] != 0:
        return True, state[0]
    if state[2] == state[4] and state[4] == state[6] and state[2] != 0:
        return True, state[2]
    for i in range(9):
        if state[i] == 0:
            return False, 0
    return True, 0

def bind(state):
    getstr = str()
    for x in state:
        getstr += str(x)
    return getstr

def available_action(state):
    actionlist = []
    for i in range(9):
        if state[i] == 0:
            actionlist.append(i)
    return actionlist

def order(state):
    c1 = c2 = 0
    
    for x in state:
        if x == 1:
            c1 += 1
        elif x == 2:
            c2 += 1
    if c1 > c2:
        go = 2
    else:
        go = 1

    return go


def following(state, action):
    state2 = copy.copy(state)
    go = order(state)

    state2[action] = go
    done, winner = end_check(state2)

    reward = 0
    if done and winner == 1: reward = 1
    if done and winner == 2: reward = -1
    if done and winner == 0: reward = 0

    return state2, reward



### Human player

In [3]:
class Human_player:    
    def __init__(self):
        self.name = "Human player"
        
    def policy(self, state):
        available = available_action(state)

        while True:
            #가능한 행동을 조사한 후 표시
            print("possible actions = ",available)
            # 상태 번호 표시
            print("+----+----+----+")
            print("+  0 +  1 +  2 +")
            print("+----+----+----+")
            print("+  3 +  4 +  5 +")
            print("+----+----+----+")
            print("+  6 +  7 +  8 +")
            print("+----+----+----+")
            getstr = int(input("Select action(human) : "))
            if getstr in available:
                return getstr
            else:
                print("You selected wrong action")
        return

### 정책 반복 플레이어

In [4]:
class Policy_set:
    def policy(self, state, go):
        available = available_action(state)
        action_list = []

        for i in available:
            state[i] = go
            done, winner = end_check(state)
            state[i] = 0
            if done:
                action_list.append(i)
        if len(action_list) == 0:
            action_list = available

        return random.choice(action_list)

class Policy_player:
    def __init__(self, rs_file=False):
        self._value = dict()
        self._policy = dict()
        self.name = "Policy player"

        if not rs_file:
            self.init_value()
        else:
            self.rs_file()

    def init_value(self):
        statelist = itertools.product([0, 1, 2], repeat=9)

        for state in statelist:
            state = list(state)
            binded = bind(state)
            done, winner = end_check(state)
            if not done:
                self._value[binded] = random.uniform(-0.5, 0.5)
                self._policy[binded] = random.choice(available_action(state))
            # terminal state value
            else:
                self._value[binded] = 0
                
    def file_save(self):
        file = open("v.dat", 'wb')
        cPickle.dump(self._value, file)
        file = open("p.dat", 'wb')
        cPickle.dump(self._policy, file)

    def rs_file(self):
        file = open("v.dat", 'rb')
        self._value = cPickle.load(file)
        file = open("p.dat", 'rb')
        self._policy = cPickle.load(file)

    def value(self, state):
        binded = bind(state)
        return self._value[binded]

    def renderv(self, state, x):
        binded = bind(state)
        self._value[binded] = x

    def policy(self, state):
        binded = bind(state)
        return self._policy[binded]

    def renderp(self, state, x):
        binded = bind(state)
        self._policy[binded] = x

episode = 100
gamma = 0.9

env = Environment()
agent = Policy_player(rs_file=False)
strucag = Policy_set()

iteration_arr = []
v_arr = []
wr_arr = []

def policyeval(agent):
    theta = 1e-9

    while True:
        delta = 0.0

        statelist = itertools.product([0, 1, 2], repeat=9)
        for state in statelist:
            state = list(state)
            done, winner = end_check(state)
            if not done:  # except for terminal state
                v = agent.value(state)

                action = agent.policy(state)
                state2, reward = following(state, action)
                agent.renderv(state, reward + gamma * agent.value(state2))

                delta = max([delta, abs(v - agent.value(state))])

        if delta < theta:
            break


def polcyimp(agent):
    stable = True

    statelist = itertools.product([0, 1, 2], repeat=9)
    for state in statelist:
        state = list(state)
        done, winner = end_check(state)
        if not done:  # except for terminal state
            available = available_action(state)
            go = order(state)

            preaction = agent.policy(state)

            valuemax = -9999999
            valuemin = 9999999
            if go == 1:
                for action in available:
                    state2, reward = following(state, action)
                    value = reward + gamma * agent.value(state2)
                    if value > valuemax:
                        valuemax = value
                        naction = action
            else:
                for action in available:
                    state2, reward = following(state, action)
                    value = reward + gamma * agent.value(state2)
                    if value < valuemin:
                        valuemin = value
                        naction = action

            agent.renderp(state, naction)

            if preaction != naction:
                stable = False

    if stable:
        return True
    else:
        return False


def policytrain():
    iteration = 0

    while True:
        iteration += 1
        avg = 0.0

        # policy iteration
        policyeval(agent)
        stop = polcyimp(agent)

        # verification stage
        win = lose = draw = 0
        tcount = 0
        for i in range(episode):
            done = 0
            env.setting()
            state = copy.copy(env.state)

            winner = 0
            j = 0
            while not done:
                j += 1
                go = copy.copy(env.go)
                if (i + j) % 2 == 1:
                    action = agent.policy(state)
                    avg += agent.value(state) * ((-1) ** (go + 1))
                    tcount += 1
                else:
                    action = strucag.policy(state, go)
                state2, done, reward, winner = env.set_reward(action)
                state = copy.copy(state2)

            if winner == 0:
                draw += 1
            elif (i + j) % 2 == 1:
                win += 1
            else:
                lose += 1
        rwin = (win + draw) / episode
        avg /= tcount

        agent.file_save()
        iteration_arr.append(iteration)
        v_arr.append(avg)
        wr_arr.append(rwin)

        if stop:
            break


if __name__ == "__main__":
    policytrain()

### 게임 진행 함수

In [5]:
env = Environment()

p1 = Human_player()
p2 = Policy_player(rs_file=True)

# 각 플레이어의 승리 횟수를 저장
p1_score = 0
p2_score = 0
draw_score = 0

print("pl player : {}".format(p1.name))
print("p2 player : {}".format(p2.name))

while True:
    done = 0
    winner = 0
    env.setting()
    state = copy.copy(env.state)

    i = 0
    while not done:
        i += 1
        if i % 2 == 1:
            action = p1.policy(state)
        else:
            action = p2.policy(state)
        state2, done, reward, winner = env.set_reward(action)
        state = copy.copy(state2)
        env.print_board()

    if reward == 0:
        print("draw")
        draw_score += 1
    elif reward == 1:
        print("winner is p1({})".format(p1.name))
        p1_score += 1
    else :
        print("winner is p2({})".format(p2.name))
        p2_score += 1
        
    # 최종 결과 출력        
    print("final result")
    env.print_board()

   # 한게임 더?최종 결과 출력 
    answer = input("More Game? (y/n)")
    if answer == 'n':
        break        

print("p1({}) = {} p2({}) = {} draw = {}".format(p1.name, p1_score,p2.name, p2_score,draw_score))
                


pl player : Human player
p2 player : Policy player
possible actions =  [0, 1, 2, 3, 4, 5, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 3
+----+----+----+
|    |    |    |
+----+----+----+
|  O |    |    |
+----+----+----+
|    |    |    |
+----+----+----+
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |    |    |
+----+----+----+
|    |    |    |
+----+----+----+
possible actions =  [1, 2, 4, 5, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  4 +  5 +
+----+----+----+
+  6 +  7 +  8 +
+----+----+----+
Select action(human) : 5
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |    |  O |
+----+----+----+
|    |    |    |
+----+----+----+
+----+----+----+
|  X |    |    |
+----+----+----+
|  O |  X |  O |
+----+----+----+
|    |    |    |
+----+----+----+
possible actions =  [1, 2, 6, 7, 8]
+----+----+----+
+  0 +  1 +  2 +
+----+----+----+
+  3 +  